In [ ]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
from collections.abc import MutableMapping
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar
import time
import re


#Logger
logging.basicConfig(filename='Anàlisi-ByTweetType.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
##############
# PARAMETERS #
##############

client = MongoClient(username='XXX', password='XXX')

DatabaseName = "Hashtags"
TweetCollectionName = "Campanya-Interactions"

db = client[DatabaseName]
tweetCollection = db[TweetCollectionName]

In [ ]:
########################
# ADD FIELD TWEET TYPE #
########################

tweetCollection = db[TweetCollectionName]
tweets = tweetCollection.find(no_cursor_timeout=True, batch_size=1000000)

tweet_type = ""

for result in tweets:

    if 'retweeted_status' in result:
        tweet_type = "RT"
    elif result['is_quote_status']:
        if 'quoted_status' in result:
            tweet_type = "QUOTE"
    elif result['in_reply_to_status_id'] is not None:
        tweet_type = "REPLY"
    else:
        tweet_type = "?"
    
    try:
        tweetCollection.update_one(
                                    {'_id': result['_id']},
                                    {'$set': 
                                        {
                                            'tweet_type': tweet_type,
                                        }
                                    },
                                    upsert=False,
                                  )
    except Exception as e:
        logging.error(e)
        logging.error("Fatal exception inserting users in MongoDB")
            
        
tweets.close() 

In [ ]:
###################################
# ANALYZE TWEET TYPE BY COMMUNITY #
###################################

def load_tweets(collection):
    """Extracts the tweet bot interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_community': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_community':'$ACTIVE_community', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(tweetCollection)
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).to_string(index=False))

tweets_df3 = tweets_df2.groupby(['ACTIVE_community', 'tweet_type']).agg({'count': 'sum'})
tweets_df4 = tweets_df3.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

print(tweets_df4.sort_values('ACTIVE_community'))

In [ ]:
#######################
# TWEETS BY COMMUNITY #
#######################

def load_tweets(collection):
    """Extracts the tweet bot interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_community': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_community':'$ACTIVE_community'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(tweetCollection)
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).to_string(index=False))

In [ ]:
################################################
# EXTRACT COMMUNITY INTERACTIONS BY TWEET TYPE #
################################################

def load_tweets(collection):
    """Extracts the tweet bot and community interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_community': True,
                        'PASSIVE_community': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_community':'$ACTIVE_community', 'PASSIVE_community':'$PASSIVE_community', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(db[TweetCollectionName])
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).head(100).to_string(index=False))

In [ ]:
##############################
# ANALYZE TWEET TYPE BY BOTS #
##############################

def load_tweets(collection):
    """Extracts the tweet bot interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_IS_BOT': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_IS_BOT':'$ACTIVE_IS_BOT', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(tweetCollection)
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).to_string(index=False))

tweets_df3 = tweets_df2.groupby(['ACTIVE_IS_BOT', 'tweet_type']).agg({'count': 'sum'})
tweets_df4 = tweets_df3.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

print(tweets_df4.sort_values('ACTIVE_IS_BOT'))

In [ ]:
##############################
# ANALYZE TWEET TYPE BY BOTS #
##############################

def load_tweets(collection):
    """Extracts the tweet bot interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_IS_BOT': True,
                        'PASSIVE_IS_BOT': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_IS_BOT':'$ACTIVE_IS_BOT', 'PASSIVE_IS_BOT':'$PASSIVE_IS_BOT', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(tweetCollection)
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).to_string(index=False))

tweets_df3 = tweets_df2.groupby(['ACTIVE_IS_BOT', 'PASSIVE_IS_BOT', 'tweet_type']).agg({'count': 'sum'})
tweets_df4 = tweets_df3.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

print(tweets_df4.sort_values('ACTIVE_IS_BOT'))

In [ ]:
################################################
# EXTRACT COMMUNITY INTERACTIONS BY TWEET TYPE #
################################################

def load_tweets(collection):
    """Extracts the tweet bot and community interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$project': {
                        'ACTIVE_community': True,
                        'PASSIVE_community': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_community':'$ACTIVE_community', 'PASSIVE_community':'$PASSIVE_community', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(db[TweetCollectionName])
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).head(100).to_string(index=False))

In [ ]:
########################################################
# EXTRACT BOT INTERACTIONS BY COMMUNITY AND TWEET TYPE #
########################################################

def load_tweets(collection):
    """Extracts the tweet bot and community interaction information
    
    Keyword arguments:
    collection -- MongoDB Tweets' Collection
    """
    pipeline = [
                {
                    '$match': {
                        'ACTIVE_IS_BOT': 'BOT'
                    }
                },{
                    '$project': {
                        'ACTIVE_IS_BOT': True,
                        'PASSIVE_IS_BOT': True,
                        'ACTIVE_community': True,
                        'PASSIVE_community': True,
                        'tweet_type': True
                    }
                }, {
                    '$group': {
                        '_id': {'ACTIVE_IS_BOT':'$ACTIVE_IS_BOT', 'PASSIVE_IS_BOT':'$PASSIVE_IS_BOT', 
                                'ACTIVE_community':'$ACTIVE_community', 'PASSIVE_community':'$PASSIVE_community', 'tweet_type':'$tweet_type'}, 
                        'count': {
                            '$sum': 1
                        }
                    }
                }
            ]
    print("Query", end=" ")
    tweets = collection.aggregate(pipeline, allowDiskUse=True)
    print("OK; List", end=" ")
    tweets = list(tweets)
    print("OK; Total combinations:", len(tweets))
    return tweets

tweets = load_tweets(db[TweetCollectionName])
tweets_df = pd.DataFrame(tweets)
tweets_df2 = pd.concat([pd.json_normalize(tweets_df['_id']), tweets_df['count']], axis=1)

totalTweets = tweets_df2['count'].sum()
print("TOTAL TWEETS: " + str(totalTweets))

print(tweets_df2.sort_values('count', ascending=False).head(100).to_string(index=False))